# Data Cleaning Notebook

## Notebook Summary
In this Notebook:
- All columns related to ADHD are removed
- All NaN values present in the dataset are addressed
- A train-test split is performed on the data, and a validation set is created
- transformed/cleaned data is saved as CSV files.

In [1]:
# Relevant imports
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

In [2]:
# Loading data into pandas dateframe
nsch = pd.read_sas('../Data/nsch_2020_topical_SAS/nsch_2020_topical.sas7bdat')

# Visually checking successful loading of dataframe
nsch.head()

/Users/matthewturner/opt/anaconda3/envs/Capstone/lib/python3.9/site-packages/pandas/io/sas/sas7bdat.py:800: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  rslt[name] = self._byte_chunk[jb, :].view(dtype=self.byte_order + "d")


,FIPSST,STRATUM,HHID,FORMTYPE,TOTKIDS_R,TENURE,HHLANGUAGE,SC_AGE_YEARS,SC_SEX,K2Q35A_1_YEARS,...,BIRTH_YR_F,BMICLASS,HHCOUNT_IF,FPL_I1,FPL_I2,FPL_I3,FPL_I4,FPL_I5,FPL_I6,FWC
0,b'17',b'1',b'20000003',b'T1',2.0,1.0,1.0,3.0,1.0,NaN,...,0.0,NaN,0.0,400.0,400.0,400.0,400.0,400.0,400.0,3296.080092
1,b'29',b'2A',b'20000004',b'T3',1.0,1.0,1.0,14.0,2.0,NaN,...,0.0,2.0,0.0,400.0,400.0,400.0,400.0,400.0,400.0,2888.545330
2,b'47',b'1',b'20000005',b'T1',1.0,1.0,1.0,1.0,2.0,NaN,...,0.0,NaN,0.0,400.0,400.0,400.0,400.0,400.0,400.0,1016.682730
3,b'28',b'1',b'20000014',b'T3',2.0,1.0,1.0,15.0,2.0,NaN,...,0.0,2.0,0.0,143.0,143.0,143.0,143.0,143.0,143.0,1042.091065
4,b'55',b'1',b'20000015',b'T3',2.0,2.0,1.0,16.0,2.0,NaN,...,0.0,3.0,0.0,400.0,400.0,400.0,400.0,400.0,400.0,402.372392


## ADHD Columns

Along with our target column, 'K2Q31A' there are multiple other columns that relate to ADHD that need to be removed. These columns are closely tied to ADHD so they need to be removed so the model doesn't take them into account when making predictions. Column descriptions can be found in the [EDA notebook](https://github.com/austint1121/Undiagnosed-ADHD-Identification/blob/main/Notebooks/EDA.ipynb). We will also save our target column, and remove any rows with null values in that column

In [3]:
# Creating list of columns to be dropped
related_ADHD = [
    'K2Q31A',
    'K2Q31B',
    'K2Q31C',
    'K2Q31D',
    'K4Q23',
    'SC_K2Q10',
    'SC_K2Q11',
    'SC_K2Q12',
    'ADDTREAT',
    'SC_CSHCN',
    'SC_K2Q22',
    'SC_K2Q10',
    'K4Q22_R',
    'K6Q15',
    'SC_K2Q20',
    'K4Q36',
    'TOTNONSHCN',
    'K4Q28X04',
]

In [4]:
# Dropping rows with NAN values in target column
dropped_adhd = nsch.dropna(subset=['K2Q31A'])

# Saving Target column
target = dropped_adhd['K2Q31A']

# Creating new dataframe without columns from above
dropped_adhd = dropped_adhd.drop(columns=related_ADHD)

# Confirming expected results
dropped_adhd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42589 entries, 0 to 42776
Columns: 426 entries, FIPSST to FWC
dtypes: float64(422), object(4)
memory usage: 138.7+ MB


### Binarizing Target Values
Currently in our target column a "1" means that the child has been diagnosed with ADHD and a 2 means they haven't been. This will cause problems down the road later, so I'm going to manually replace them with 0's and 1's.

In [5]:
# Dictionary for replacement
testing = {2: 0, 1: 1}

# Preforming replacement
target = target.replace(testing)

# Visually confirming expected change
target.value_counts()


0.0    38283
1.0     4306
Name: K2Q31A, dtype: int64

### Dropping Object Columns

In [6]:
# Checking the column types
dropped_adhd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42589 entries, 0 to 42776
Columns: 426 entries, FIPSST to FWC
dtypes: float64(422), object(4)
memory usage: 138.7+ MB


In [7]:
# We have 422 float64 and 4 object types. Lets investigate those 4 objects
dropped_adhd.select_dtypes('object')

,FIPSST,STRATUM,HHID,FORMTYPE
0,b'17',b'1',b'20000003',b'T1'
1,b'29',b'2A',b'20000004',b'T3'
2,b'47',b'1',b'20000005',b'T1'
3,b'28',b'1',b'20000014',b'T3'
4,b'55',b'1',b'20000015',b'T3'
...,...,...,...,...
42772,b'26',b'1',b'20239975',b'T3'
42773,b'54',b'1',b'20239979',b'T2'
42774,b'54',b'1',b'20239980',b'T3'
42775,b'15',b'1',b'20239994',b'T2'


**FIPSST** - State FIPS code
**STRATUM** - Sampling Stratum
**HHID** - Unique Household ID
**FORMTYPE** - A proxy for age, kids are given a form base on age ranges (T1: 0-5, T2: 6-11, T3:12-17)

All of these columns can be dropped as they should have an effect on whether a child has ADHD, or they are a proxy for an already present variable.

In [8]:
# Dropping object columns
dropped_final = dropped_adhd.drop(columns=['FIPSST', 'STRATUM', 'HHID', 'FORMTYPE'])
# Confirming expected column count
dropped_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42589 entries, 0 to 42776
Columns: 422 entries, TOTKIDS_R to FWC
dtypes: float64(422)
memory usage: 137.4 MB


## Train Test Split
Before doing any transformations it will be necessary to perform the train test split beforehand to prevent data leakage.

In [9]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(dropped_final, target, random_state=15, stratify=target)

# Split test into a testing and final holdout/validation set
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, random_state=15, stratify=y_test)

# Printing total rows in each set
print(f'Training set is {len(X_train)} entries')
print(f'Testing set is {len(X_test)} entries')
print(f'Validation set is {len(X_val)} entries')

Training set is 31941 entries
Testing set is 7986 entries
Validation set is 2662 entries


In [10]:
# Printing the amount of kids diagnosed with ADHD in each split
print(f'There are {y_train.value_counts().values[1]} kids with ADHD in the training set')
print(f'There are {y_test.value_counts().values[1]} kids with ADHD in the testing set')
print(f'There are {y_val.value_counts().values[1]} kids with ADHD in the validation set')

There are 3229 kids with ADHD in the training set
There are 808 kids with ADHD in the testing set
There are 269 kids with ADHD in the validation set



## Handling Missing Values
There are multiple strategies to handling missing values, normally I would love to use Sklearns experimental [Iterative Imputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html) however, in this survey some questions are sub questions of others, and can be left blank as a result of the answer to the parent question.
<br>
An example of this is the "ADDTREAT" column we dropped earlier. This column is left blank if "K2Q31A" (our target) is answered "No". For now, I will create an imputer but not use it yet, as it would take a large amount of time for it to run on the large dataset.

In [11]:
# Simple Imputer filling values with something that vill obviously be a nan value, in this case, 999
SI_imputer = SimpleImputer(strategy='constant', fill_value=999)

In [12]:
# Fitting the imputer to training data / transforming the training data
transformed_X_train = SI_imputer.fit_transform(X_train, y_train)

# Creating a dataframe and re-adding the column names to the data
transformed_X_train = pd.DataFrame(transformed_X_train, columns=X_train.columns)

# Visually confirming transformation
transformed_X_train.head()

,TOTKIDS_R,TENURE,HHLANGUAGE,SC_AGE_YEARS,SC_SEX,K2Q35A_1_YEARS,BIRTH_MO,BIRTH_YR,MOMAGE,K6Q41R_STILL,...,BIRTH_YR_F,BMICLASS,HHCOUNT_IF,FPL_I1,FPL_I2,FPL_I3,FPL_I4,FPL_I5,FPL_I6,FWC
0,1.0,1.0,1.0,8.0,2.0,999.0,10.0,2011.0,39.0,999.0,...,0.0,999.0,0.0,400.0,400.0,400.0,400.0,400.0,400.0,2691.769960
1,2.0,1.0,1.0,17.0,1.0,999.0,4.0,2003.0,25.0,999.0,...,0.0,4.0,1.0,400.0,400.0,400.0,400.0,319.0,400.0,3094.739915
2,2.0,1.0,1.0,16.0,1.0,2.0,7.0,2004.0,34.0,999.0,...,0.0,4.0,1.0,249.0,170.0,400.0,400.0,244.0,372.0,1557.804645
3,2.0,1.0,1.0,1.0,1.0,999.0,8.0,2019.0,36.0,999.0,...,0.0,999.0,0.0,231.0,231.0,231.0,231.0,231.0,231.0,8553.637678
4,4.0,2.0,1.0,16.0,2.0,999.0,5.0,2004.0,40.0,999.0,...,0.0,2.0,0.0,223.0,223.0,223.0,223.0,223.0,223.0,106.781469


In [13]:
# Transforming test and validation set
transformed_X_test = SI_imputer.transform(X_test)

# Creating a dataframe and re-adding the column names to the data
transformed_X_test= pd.DataFrame(transformed_X_test, columns=X_test.columns)

# Transforming Validation set
transformed_X_val = SI_imputer.transform(X_val)

# Creating a dataframe and re-adding the column names to the data
transformed_X_val= pd.DataFrame(transformed_X_val, columns=X_val.columns)


## Conclusion

In [14]:
# Saving the transformed dataframes

# Training Data
transformed_X_train.to_csv('../Data/train/X_train.csv')
y_train.to_csv('../Data/train/y_train.csv')

# Testing Data
transformed_X_test.to_csv('../Data/test/X_test.csv')
y_test.to_csv('../Data/test/y_test.csv')

# Validation Data
transformed_X_val.to_csv('../Data/val/X_val.csv')
y_val.to_csv('../Data/val/y_val.csv')

From this point forward, any more cleaning would be model specific. I could also clean the data more to improve the overall quality, but I think this will be a good "baseline" starting point for the modeling process. The next step is to begin the [modeling proccess](Modeling/First_Simple_Models.ipynb)